<b>Prueba #6: Gestión de datos con ElasticSearch y visualización con Matplotlib

<b>1.1 En un histograma o en un line plot, visualizar el número de tweets conteniendo la palabra “goal”, cada minuto.


In [ ]:
#CREAR UN INDEX PARA ALMACENAR LOS DATOS
from elasticsearch import Elasticsearch
from pprint import pprint

es = Elasticsearch()

if not(es.indices.exists(index="tp6")):
    res = es.indices.create(index="tp6")  
    pprint(res)

In [ ]:
#LISTAR LOS ARCHIVOS DEL DATASET
import json
import os
files = os.listdir("dataset_tp6")
pprint(len(files))

In [ ]:
from elasticsearch import helpers

#CARGAR LOS DATOS EN ELASTICSEARCH
tweets=[]

for filename in files:
    with open('dataset_tp6/'+filename) as f:
        try:
            #pprint(filename+':')  timestamp, text, username y location
            tweet = json.load(f)
            timestamp=int(tweet["timestamp_ms"])
            #pprint(tweet)
            simple_tweet = {
                "timestamp":timestamp, \
               "text": tweet['text'], \
               "username": tweet['user']['name'], \
               "location": tweet['user']['location']}
        except KeyError:
            error="error"
            
    tweets.append(simple_tweet)#remove comment
    
    #CARGAMOS LOS DATOS EN ELASTICSEARCH CON CONSULTAS 'BULK' PARA OPTIMIZAR EL PROCESO
    if (len(tweets)>=1000):
        i=0
        docs_as_string="["
        while i<1000:
            if i!=0:
                docs_as_string = docs_as_string + ','
            docs_as_string = docs_as_string + '\n' + json.dumps(tweets[i])
            i=i+1
        docs_as_string=docs_as_string+"]"
        res = helpers.bulk(es, json.loads(docs_as_string),index="tp6", doc_type="_doc")
        tweets=[]

In [ ]:
#CONSULTA: ¿Cuántos tweets cada minuto?
res = es.search(index="tp6", body={"aggs" : { "tweets_over_time" : { "date_histogram" : { "field" : "timestamp",
                                                                                          "interval" : "1m"
                                                                                         },
      "aggs" : {
    "texto_filtro" : {
      "filters" : {
        "filters" : {
          "cantidad" :   { "match" : { "text" : "goal"   }},
        }
      }
    }
  }
                                                            }
                                            }
                                  }
               )
for data in res['aggregations']['tweets_over_time']['buckets']:
    print(data['doc_count'])
    #data['texto_filtro']['buckets']['cantidad']['doc_count']
    

In [ ]:
import pandas as pd

In [ ]:
df_tweetsByMinutes2 = pd.Series([data['texto_filtro']['buckets']['cantidad']['doc_count']])
i = 1
for data in res['aggregations']['tweets_over_time']['buckets']:
    print(data['texto_filtro']['buckets']['cantidad']['doc_count'])
    df_tweetsByMinutes2[i] = data['texto_filtro']['buckets']['cantidad']['doc_count']
    i = i + 1

In [ ]:
df_tweetsByMinutes2[:10]

In [ ]:
df_tweetsByMinutes = pd.DataFrame(res['aggregations']['tweets_over_time']['buckets'])
df_tweetsByMinutes = df_tweetsByMinutes.rename(columns={'key': 'timestamp', 'doc_count': 'nb_tweets_old'})
df_tweetsByMinutes[:10]

In [ ]:
tweets_goal = df_tweetsByMinutes[['timestamp']].copy()

tweets_goal.loc[:,'numeroTweets'] = df_tweetsByMinutes2
tweets_goal[:10]


In [ ]:
#df_tweetsByMinutes_filtered = df_tweetsByMinutes.drop(df_tweetsByMinutes[df_tweetsByMinutes.timestamp < 1527357600000 ].index)
#df_tweetsByMinutes_filtered[:10]

In [ ]:
import datetime

def timestamp2date(x):
    y = datetime.datetime.fromtimestamp(x/1000).strftime('%H:%M:%S')
    return y

tweets_goal['timestamp']=tweets_goal['timestamp'].apply(timestamp2date)
tweets_goal[:10]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

tweets_goal.plot(kind='bar',figsize=(40,10), fontsize=10, x='timestamp')
plt.show()

<b>1.2 En un histograma o en un line plot, visualizar el número de tweets conteniendo la palabra “goal”, cada 10 minutos.



In [ ]:
#CONSULTA: ¿Cuántos tweets cada minuto?
res2 = es.search(index="tp6", body={"aggs" : { "tweets_over_time" : { "date_histogram" : { "field" : "timestamp",
                                                                                          "interval" : "10m"
                                                                                         },
      "aggs" : {
    "texto_filtro" : {
      "filters" : {
        "filters" : {
          "cantidad" :   { "match" : { "text" : "goal"   }},
        }
      }
    }
  }
                                                            }
                                            }
                                  }
               )
for data2 in res2['aggregations']['tweets_over_time']['buckets']:
    print(data2['doc_count'])
    #data['texto_filtro']['buckets']['cantidad']['doc_count']
    

In [ ]:
df_tweetsByMinutes3 = pd.Series([data2['texto_filtro']['buckets']['cantidad']['doc_count']])
i = 1
for data2 in res2['aggregations']['tweets_over_time']['buckets']:
    print(data2['texto_filtro']['buckets']['cantidad']['doc_count'])
    df_tweetsByMinutes3[i] = data2['texto_filtro']['buckets']['cantidad']['doc_count']
    i = i + 1

In [ ]:
df_tweetsByMinutes3[:10]

In [ ]:
df_tweetsByMinutes4 = pd.DataFrame(res2['aggregations']['tweets_over_time']['buckets'])
df_tweetsByMinutes4 = df_tweetsByMinutes4.rename(columns={'key': 'timestamp', 'doc_count': 'nb_tweets_old'})
df_tweetsByMinutes4[:10]

In [ ]:
tweets_goal2 = df_tweetsByMinutes4[['timestamp']].copy()

tweets_goal2.loc[:,'numeroTweets'] = df_tweetsByMinutes3
tweets_goal2[:10]



In [ ]:
import datetime

def timestamp2date(x):
    y = datetime.datetime.fromtimestamp(x/1000).strftime('%H:%M:%S')
    return y

tweets_goal2['timestamp']=tweets_goal2['timestamp'].apply(timestamp2date)
tweets_goal2[:10]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

tweets_goal2.plot(kind='bar',figsize=(40,10), fontsize=10, x='timestamp')
plt.show()

<b>2. En un histograma o en un line plot, visualizar dos series de datos: 
el número de tweets que hablan de ‘liverpool’ por cada minuto,
el número de tweets que hablan de ‘madrid’ por cada minuto.

            ¿A qué minuto se habla más de Madrid?


<b>2.1 Obteniendo tabla de liverpool.

In [ ]:
#CONSULTA: ¿Cuántos tweets cada minuto?
resultado = es.search(index="tp6", body={"aggs" : { "tweets_over_time" : { "date_histogram" : { "field" : "timestamp",
                                                                                          "interval" : "1m"
                                                                                         },
      "aggs" : {
    "texto_filtro" : {
      "filters" : {
        "filters" : {
          "cantidad" :   { "match" : { "text" : "liverpool"   }},
        }
      }
    }
  }
                                                            }
                                            }
                                  }
               )
for datos in resultado['aggregations']['tweets_over_time']['buckets']:
    print(datos['doc_count'])   

In [ ]:
tminuto = pd.Series([datos['texto_filtro']['buckets']['cantidad']['doc_count']])
i = 1
for datos in resultado['aggregations']['tweets_over_time']['buckets']:
    print(datos['texto_filtro']['buckets']['cantidad']['doc_count'])
    tminuto[i] = datos['texto_filtro']['buckets']['cantidad']['doc_count']
    i = i + 1

In [ ]:
tminuto[:10]

In [ ]:
tminuto2 = pd.DataFrame(resultado['aggregations']['tweets_over_time']['buckets'])
tminuto2 = tminuto2.rename(columns={'key': 'timestamp', 'doc_count': 'nb_tweets_old'})
tminuto2[:10]

In [ ]:
tweets_liverpool = tminuto2[['timestamp']].copy()
tweets_liverpool.loc[:,'numeroTweets'] = tminuto
tweets_liverpool[:10]

In [ ]:
import datetime

def timestamp2date(x):
    y = datetime.datetime.fromtimestamp(x/1000).strftime('%H:%M:%S')
    return y

tweets_liverpool['timestamp']=tweets_liverpool['timestamp'].apply(timestamp2date)
tweets_liverpool[:10]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

tweets_liverpool.plot(kind='bar',figsize=(40,10), fontsize=10, x='timestamp')
plt.show()

<b>2.2 Obteniendo tabla de madrid.

In [ ]:
#CONSULTA: ¿Cuántos tweets cada minuto?
resultado_m = es.search(index="tp6", body={"aggs" : { "tweets_over_time" : { "date_histogram" : { "field" : "timestamp",
                                                                                          "interval" : "1m"
                                                                                         },
      "aggs" : {
    "texto_filtro" : {
      "filters" : {
        "filters" : {
          "cantidad" :   { "match" : { "text" : "madrid"   }},
        }
      }
    }
  }
                                                            }
                                            }
                                  }
               )
for datos_m in resultado_m['aggregations']['tweets_over_time']['buckets']:
    print(datos_m['doc_count'])   

In [ ]:
tminuto_m = pd.Series([datos_m['texto_filtro']['buckets']['cantidad']['doc_count']])
i = 1
for datos_m in resultado_m['aggregations']['tweets_over_time']['buckets']:
    print(datos_m['texto_filtro']['buckets']['cantidad']['doc_count'])
    tminuto_m[i] = datos_m['texto_filtro']['buckets']['cantidad']['doc_count']
    i = i + 1

In [ ]:
tminuto_m[:10]

In [ ]:
tminuto2_m = pd.DataFrame(resultado_m['aggregations']['tweets_over_time']['buckets'])
tminuto2_m = tminuto2_m.rename(columns={'key': 'timestamp', 'doc_count': 'nb_tweets_old'})
tminuto2_m[:10]

In [ ]:
tweets_madrid = tminuto2_m[['timestamp']].copy()
tweets_madrid.loc[:,'numeroTweets'] = tminuto_m
tweets_madrid[:10]

In [ ]:
import datetime

def timestamp2date(x):
    y = datetime.datetime.fromtimestamp(x/1000).strftime('%H:%M:%S')
    return y

tweets_madrid['timestamp']=tweets_madrid['timestamp'].apply(timestamp2date)
tweets_madrid

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

tweets_madrid.plot(kind='bar',figsize=(40,10), fontsize=10, x='timestamp')
plt.show()

<b>Gráficos de liverpool y madrid conbinados

In [ ]:
tweets_final = tweets_liverpool[['timestamp','numeroTweets']].copy()
tweets_final = tweets_final.rename(columns={'timestamp': 'hora', 'numeroTweets': 'liberpool'})
#tweets_madrid = tweets_madrid.drop('timestamp', 1)
tweets_final.loc[:,'numeroTweets'] = tweets_madrid
tweets_final = tweets_final.rename(columns={'numeroTweets': 'madrid'})
tweets_final[:10]

In [ ]:
#Ahora podemos visualizar el número de medallas por cada tipo de medalla y cada país
tweets_final.plot(kind='bar',figsize=(40,10), fontsize=20, x='hora', y=['liberpool','madrid'])
plt.show()

<b>¿a qué minuto se habla más de madrid?


<b>RESPUESTA: AL MINUTO 16:40:00